<a href="https://colab.research.google.com/github/htapiagroup/sistemas-distribuidos-evaluacion-intersemestral-asosam91/blob/master/MCA_SistDist_Ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adrián Sosa Medellín
## MCA 2018-2020

### Configuracion inicial

In [0]:
import os, time, psutil, gc
import pandas as pd 
from google.colab import drive

In [2]:
drive.mount('/content/gdrive');

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
cd gdrive/My\ Drive/Colab\ Notebooks/Data

/content/gdrive/My Drive/Colab Notebooks/Data


In [0]:
gc.enable()

### definicion de funciones

In [0]:
# esta funcion fue proporcionada por el Dr. Horacio Tapia
def memoryPrint():
  '''Regresa al memoria usada por un proceso en python'''
  mem = psutil.Process(os.getpid()).memory_info().rss
  return (mem/1024**2)

In [0]:
class data():
  def __init__(self):
    self.name = ""
    self.rows = 0
    self.cols = 0
    self.readtime = 0
    self.memoryprint = 0 
    self.memory = 0
    self.empyreg = 0
    self.newcols = 0
    self.newrows = 0
    self.timemean = 0

### descarga y descompresion de la 'Base de datos'

In [0]:
nombre = 'trip_data_{:d}.csv'
plantilla = [nombre.format(i) for i in range(1, 13)]

In [0]:
# if not os.path.exists('trip_data.7z'):
#   comando = "wget https://archive.org/download/nycTaxiTripData2013/trip_data.7z"
#   os.system(comando)

# !7z x trip_data.7z

In [0]:
resultados = []

In [0]:
# no ejecutar esta celda a menos que sea necesario
# for i in plantilla:
#   comando = "rm new_"+ i
#   os.system(comando)

In [0]:
for archivo in plantilla:
  nombre = "new_" + archivo
  if not os.path.exists(nombre):
    dat = data()
    m1 = memoryPrint()
    t1 = time.time()
    try:
      df = pd.read_csv(archivo, usecols=["medallion",  "pickup_datetime", "dropoff_datetime", "passenger_count", "trip_time_in_secs", "trip_distance", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"])
    except :
      pass
    try:
      df = pd.read_csv(archivo, usecols=["medallion",  " pickup_datetime", " dropoff_datetime", " passenger_count", " trip_time_in_secs", " trip_distance", " pickup_longitude", " pickup_latitude", " dropoff_longitude", " dropoff_latitude"])
    except :
      pass
        
    t2 = time.time()
    m2 = memoryPrint()
    dat.name = archivo
    dat.readtime = (t2 - t1)*1000
    dat.memoryprint = m2 - m1
    dat.memory = df.memory_usage().sum()
    shapes = df.shape
    dat.rows = shapes[0]
    dat.cols = shapes[1]
    
    df = df.rename(columns=lambda x: x.strip())
    dat.empyreg = df.isna().sum()
    df.dropna()
    shapes = df.shape
    dat.newrows = shapes[0]
    dat.newcols = shapes[1]
    t1 = time.time()
    mean = df["trip_distance"].mean()
    t2 = time.time()
    dat.timemean = (t2 - t1)*1000
    df.to_csv(nombre)
    resultados.append(dat)
    # comando = "rm " + archivo
    # os.system(comando)
    df = ''
    dat = ''

In [0]:
# muestro el resultado de las operaciones
print("Resultados de las operaciones con PANDAS")

for i in range(len(resultados)):
  print("\n")
  print("Archivo analizado: {}".format(resultados[i].name))
  print("El archivo tiene {} columnas y {} renglones".format(resultados[i].cols, resultados[i].rows))
  print("Numero de registros vacios:\n{}".format(resultados[i].empyreg))
  print("--- se crea un nuevo conjunto de datos con los errores eliminados si los hay. ---")
  print("El nuevo conjunto de datos tiene {} columnas y {} renglones".format(resultados[i].newcols, resultados[i].newrows))
  print("Tiempo de lectura del archivo {}: {} ms".format(resultados[i].name, resultados[i].readtime))
  print("Huella de memoria del archivo: {:.2f} Mb".format(resultados[i].memoryprint))
  print("Memoria del sistema usada para crear el archivo: {:.2f} Mb".format((resultados[i].memory)/(1024**2)))
  print("Tiempo que tarda pandas en obtener el promedio de Trip_distance: {} ms".format(resultados[i].timemean))
  print("-----------------------------------------------------------------------------------------")
  

Resultados de las operaciones con PANDAS


Archivo analizado: trip_data_1.csv
El archivo tiene 10 columnas y 14776615 renglones
Numero de registros vacios:
medallion             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
trip_time_in_secs     0
trip_distance         0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude    86
dropoff_latitude     86
dtype: int64
--- se crea un nuevo conjunto de datos con los errores eliminados si los hay. ---
El nuevo conjunto de datos tiene 10 columnas y 14776615 renglones
Tiempo de lectura del archivo trip_data_1.csv: 64223.88029098511 ms
Huella de memoria del archivo: 3195.27 Mb
Memoria del sistema usada para crear el archivo: 1127.37 Mb
Tiempo que tarda pandas en obtener el promedio de Trip_distance: 28.361082077026367 ms
-----------------------------------------------------------------------------------------


Archivo analizado: trip_data_2.csv
El archivo tiene 10 columnas y 13990176 renglones
Numero de

## DASK

In [0]:
import dask
import dask.dataframe as dd

In [0]:
df = dd.read_csv("new_trip_data_*.csv", usecols= ["trip_distance","trip_time_in_secs"])

In [0]:
promedio = df["trip_distance"].mean()

In [0]:
t1 = time.time()
pDist = promedio.compute()
t2 = time.time()

print("El tiempo que tarda DASK en calcular el promedio de trip_distance es: {:.2f} ms".format((t2 - t1)*1000))

In [0]:
promedio = df["trip_time_in_secs"].mean()

In [0]:
t1 = time.time()
pTime = promedio.compute()
t2 = time.time()

print("El tiempo que tarda DASK en calcular el promedio de trip_time_in_secs es: {:.2f} ms".format((t2 - t1)*1000))

In [0]:
df = dd.read_csv("new_trip_data_*.csv", usecols= ["pickup_datetime", "dropoff_datetime", "trip_time_in_secs"])

In [0]:
df["pickup_datetime"] = dd.to_datetime(df["pickup_datetime"])
df["dropoff_datetime"] = dd.to_datetime(df["dropoff_datetime"])
df["duracion"] = df["dropoff_datetime"] - df["pickup_datetime"]

In [0]:
df["duracion"] = df["duracion"].dt.total_seconds()
diferencia = df[(df["duracion"] != df["trip_time_in_secs"])]
resultado = df.compute().shape[0] - diferencia.compute().shape[0]
print("hay {} registros donde la duracion calculada y la registrada no coinciden.".format(resultado))

In [0]:
longtrip = df[df["duracion"] > 1200]

In [0]:
topCab = dd.read_csv("new_trip_data_*.csv", usecols= ["medallion", "dropoff_datetime", "trip_time_in_secs"])